# Tinder para empresas

## Imports

In [12]:
# Imports

import numpy as np
import pandas as pd

from random import shuffle

from func_matching import calculate_match
from func_args import get_args
from func_grafos import create_graph, get_best_match
from func_aquisition import adquisition_survey
from func_outputs import *
from func_normalize import *

In [13]:

'''
# funciona por ahora bien con más estudiantes que empresas
def run(G, RONDAS, student_queue, companies): 
    res = []

    for r in range(RONDAS): 
        print(r)
        aux = []
        used = []
        while len(used) < 2 * len(companies): # ahora used tiene el doble de tamaño. Hay que modificarlo

            student = student_queue.pop(0) 
            student_queue.append(student)

            # print(student)
            node1, node2, w = get_best_match(G, student, used) # creo que se queda sin opciones

            # aux.append((node1, node2, w))
            G.remove_edge(node1, node2)
            used.extend([node1, node2])


            # el nodo1 es la compañía (nodo1 no es estudiante)
            if node1 != student: 
                # used.append(node1)
                aux.append((node1, node2, w))
            # nodo2 es la compañía
            else: 
                # used.append(node2)
                aux.append((node2, node1, w))
            # print(used)
        res.append(aux)

    return res
'''

'''
# funciona creando una lista de dict

def run(G, n_rondas, student_queue, companies): 
    res = []
    for r in range(n_rondas): 
        print(r)
        aux = []
        used = []
            
        while len(used) < 2 * len(companies): # ahora used tiene el doble de tamaño. Hay que modificarlo

            student = student_queue.pop(0) 
            student_queue.append(student)

            # print(student)
            node1, node2, w = get_best_match(G, student, used) # creo que se queda sin opciones

            # aux.append((node1, node2, w))
            G.remove_edge(node1, node2)
            used.extend([node1, node2])

            if node1 not in companies: # el nodo1 es la compañía (nodo1 no es estudiante)
                print('invierte')
                node2, node1 = node1, node2
            aux.append({'ronda': r, 'company': node1, 'student': node2, 'weight': w})
            
            # print(used)
        
        res.extend(aux)
    
    print(res)
    return res
'''
def most_common(lst):
    return max(set(lst), key=lst.count)

def run(G, n_rondas, student_queue, companies): 
    res = []
    mesas = len(companies)
    for r in range(n_rondas): 
        used = []
        
        for interview in range(mesas): 
            # debería ser el studiante que más sale en student_queue
            student = most_common(student_queue) 
            
            m = get_best_match(G, student, used)
            if m == None: 
                break
            node1, node2, w = m
            G.remove_edge(node1, node2)
            used.extend([node1, node2]) # añadimos empresa y compañia para que no vuelvan a aparecer en esta ronda
            
            if node1 not in companies: # si el nodo1 no es nombre de compañía, debemos intercambiar
                node2, node1 = node1, node2
            res.append({'ronda': r, 'company': node1, 'student': node2, 'weight': w})
            student_queue.remove(node2)
    return res    

def generate_student_queue(students_list, n_rondas): 
    shuffle(students_list)
    return students_list * n_rondas



In [14]:
def main(): 
    # Toma de argumentos. 
    bootcamp, companies_filename, students_filename, n_rondas = get_args()
    
    # DFs limpios
    companies, students = adquisition_survey(bootcamp, companies_filename, students_filename)
    
    # Normalize
    students, companies = normalize_2dfs(students, companies)
    
    # TO DO: crear la tabla matching
    matching = calculate_match(students, companies, bootcamp)
    
    # Test
    #display(matching)
    
    # TO DO: en función de las rondas y el n-students y n-companies hacer los emparejamientos
    G = create_graph(matching)
    
    students_list = matching.index
    companies_list = matching.columns
    
    students_list_queue = generate_student_queue(list(matching.index), n_rondas)
    # display(students_list_queue)
    
    # display(matching.head())
    
    # vamos a recorrer la lista para ir emparejando
    rondas = run(G, n_rondas, students_list_queue, companies_list)
    #display(rondas)
    
    df = pd.DataFrame.from_dict(rondas, orient='columns')
    #display(df)
    
    df_matrix = create_matrix_ronda_empresa(rondas)
    display(df_matrix)
    
    df_matrix2 = create_matrix_ronda_estudiante(rondas)
    display(df_matrix2)

    # check
    for r in range(11): 
        ronda10a = set(df[df['ronda']==r]['student']) 
        ronda10b = set(df_matrix.iloc[r, :])
        display(ronda10a == ronda10b)
    
    # display(df.ronda.value_counts())
    # display(df.company.value_counts())
    # display(df.student.value_counts())
    # TO DO: muestra de datos en excel a gusto de A Rojas
    
    # Check
    # print(min(student_rounds.values()), max(student_rounds.values()))

if __name__== '__main__':
    main()

,InboundCycle,Bit Genoma Digital Solutions SL,Project Lobster,AKKA,C14TORCE,The Hotels Network,Atomian,Bit Genoma Digital Solutions,Farmaconnect,Volcanic Internet,SEAT,ONEBOX,Norwegian,Visyon (Estudio Bay SL),Exoticca,Bluekik,Movetia,CAPITOLE CONSULTING
0,Francisco Leston Ramos,Mario Quiroga Cuervo,Maria Isabel Llanes Agudelo,Baltasar Torres Garcia,Javier Requejo Melchor,Isabel Rodríguez Godoy,Nira Alonso Rodríguez,Thilo Sperber,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,Nerea Aparicio,Abel Colmenares Napolitano,Rodrigo Otero Montero,Alfonso Silva Feria,Enrique Anaya Marin,Moisés García Almeida,Carlos Martínez Crespo,Verónica Celemín Lominchar
1,Dan Mihai Grigoriciuc,Carlos Eduardo Lores Hernández,Patricia Defez Díaz de Sarralde,Eugenio Ruiz Cepeda,Sofia Jordan,Alfredo Hinarejos Corbeá,Sergio Ortega Gil,Daniel García Martínez,Maria Isabel Llanes Agudelo,Francisco Leston Ramos,Daniel Herrero Hernando,Roberto Muñoz Aragón,Mario Quiroga Cuervo,Javier Pérez-Chao Rodríguez,Javier Requejo Melchor,Baltasar Torres Garcia,Isabel Rodríguez Godoy,Nira Alonso Rodríguez
2,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,Nerea Aparicio,Thilo Sperber,Abel Colmenares Napolitano,Moisés García Almeida,Rodrigo Otero Montero,Enrique Anaya Marin,Carlos Martínez Crespo,Alfonso Silva Feria,Carlos Eduardo Lores Hernández,Verónica Celemín Lominchar,Dan Mihai Grigoriciuc,Eugenio Ruiz Cepeda,Patricia Defez Díaz de Sarralde,Sofia Jordan,Sergio Ortega Gil,Daniel García Martínez
3,Alfredo Hinarejos Corbeá,Daniel Herrero Hernando,Javier Pérez-Chao Rodríguez,Roberto Muñoz Aragón,Mario Quiroga Cuervo,Baltasar Torres Garcia,Francisco Leston Ramos,Maria Isabel Llanes Agudelo,Thilo Sperber,Nira Alonso Rodríguez,Isabel Rodríguez Godoy,Javier Requejo Melchor,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,Nerea Aparicio,Abel Colmenares Napolitano,Rodrigo Otero Montero,Alfonso Silva Feria
4,Carlos Martínez Crespo,Enrique Anaya Marin,Moisés García Almeida,Verónica Celemín Lominchar,Carlos Eduardo Lores Hernández,Eugenio Ruiz Cepeda,Dan Mihai Grigoriciuc,Patricia Defez Díaz de Sarralde,Alfredo Hinarejos Corbeá,Sergio Ortega Gil,Daniel García Martínez,Sofia Jordan,Daniel Herrero Hernando,Roberto Muñoz Aragón,Javier Pérez-Chao Rodríguez,Francisco Leston Ramos,Maria Isabel Llanes Agudelo,Mario Quiroga Cuervo
5,Baltasar Torres Garcia,Isabel Rodríguez Godoy,Nira Alonso Rodríguez,Javier Requejo Melchor,Thilo Sperber,Nerea Aparicio,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,Rodrigo Otero Montero,Carlos Martínez Crespo,Abel Colmenares Napolitano,Enrique Anaya Marin,Alfonso Silva Feria,Moisés García Almeida,Carlos Eduardo Lores Hernández,Verónica Celemín Lominchar,Dan Mihai Grigoriciuc,Eugenio Ruiz Cepeda
6,Sergio Ortega Gil,Patricia Defez Díaz de Sarralde,Daniel García Martínez,Sofia Jordan,Daniel Herrero Hernando,Francisco Leston Ramos,Alfredo Hinarejos Corbeá,Roberto Muñoz Aragón,Javier Requejo Melchor,Maria Isabel Llanes Agudelo,Javier Pérez-Chao Rodríguez,Baltasar Torres Garcia,Isabel Rodríguez Godoy,Mario Quiroga Cuervo,Nira Alonso Rodríguez,Thilo Sperber,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco
7,Alfonso Silva Feria,Nerea Aparicio,Rodrigo Otero Montero,Abel Colmenares Napolitano,Enrique Anaya Marin,Carlos Eduardo Lores Hernández,Moisés García Almeida,Carlos Martínez Crespo,Verónica Celemín Lominchar,Dan Mihai Grigoriciuc,Eugenio Ruiz Cepeda,Patricia Defez Díaz de Sarralde,Sofia Jordan,Sergio Ortega Gil,Daniel Herrero Hernando,Daniel García Martínez,Roberto Muñoz Aragón,Alfredo Hinarejos Corbeá
8,Javier Pérez-Chao Rodríguez,Francisco Leston Ramos,Baltasar Torres Garcia,Mario Quiroga Cuervo,Isabel Rodríguez Godoy,Nira Alonso Rodríguez,Maria Isabel Llanes Agudelo,Javier Requejo Melchor,Nerea Aparicio,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,Thilo Sperber,Abel Colmenares Napolitano,Rodrigo Otero Montero,Moisés García Almeida,Alfonso Silva Feria,Enrique Anaya Marin,Carlos Martínez Crespo
9,Verónica Celemín Lom

,Francisco Leston Ramos,Mario Quiroga Cuervo,Maria Isabel Llanes Agudelo,Baltasar Torres Garcia,Javier Requejo Melchor,Isabel Rodríguez Godoy,Nira Alonso Rodríguez,Thilo Sperber,Jaime Jacobo Rodríguez del Valle,Maria Cristina Blanco,...,Dan Mihai Grigoriciuc,Eugenio Ruiz Cepeda,Patricia Defez Díaz de Sarralde,Sofia Jordan,Sergio Ortega Gil,Daniel García Martínez,Daniel Herrero Hernando,Alfredo Hinarejos Corbeá,Roberto Muñoz Aragón,Javier Pérez-Chao Rodríguez
0,InboundCycle,Bit Genoma Digital Solutions SL,Project Lobster,AKKA,C14TORCE,The Hotels Network,Atomian,Bit Genoma Digital Solutions,Farmaconnect,Volcanic Internet,...,,,,,,,,,,
1,Volcanic Internet,Norwegian,Farmaconnect,Bluekik,Exoticca,Movetia,CAPITOLE CONSULTING,,,,...,InboundCycle,AKKA,Project Lobster,C14TORCE,Atomian,Bit Genoma Digital Solutions,SEAT,The Hotels Network,ONEBOX,Visyon (Estudio Bay SL)
2,,,,,,,,AKKA,InboundCycle,Bit Genoma Digital Solutions SL,...,Norwegian,Visyon (Estudio Bay SL),Exoticca,Bluekik,Movetia,CAPITOLE CONSULTING,,,,
3,Atomian,C14TORCE,Bit Genoma Digital Solutions,The Hotels Network,ONEBOX,SEAT,Volcanic Internet,Farmaconnect,Norwegian,Visyon (Estudio Bay SL),...,,,,,,,Bit Genoma Digital Solutions SL,InboundCycle,AKKA,Project Lobster
4,Bluekik,CAPITOLE CONSULTING,Movetia,,,,,,,,...,Atomian,The Hotels Network,Bit Genoma Digital Solutions,ONEBOX,Volcanic Internet,SEAT,Norwegian,Farmaconnect,Visyon (Estudio Bay SL),Exoticca
5,,,,InboundCycle,AKKA,Bit Genoma Digital Solutions SL,Project Lobster,C14TORCE,Atomian,Bit Genoma Digital Solutions,...,Movetia,CAPITOLE CONSULTING,,,,,,,,
6,The Hotels Network,Visyon (Estudio Bay SL),Volcanic Internet,ONEBOX,Farmaconnect,Norwegian,Exoticca,Bluekik,Movetia,CAPITOLE CONSULTING,...,,,Bit Genoma Digital Solutions SL,AKKA,InboundCycle,Project Lobster,C14TORCE,Atomian,Bit Genoma Digital Solutions,SEAT
7,,,,,,,,,,,...,Volcanic Internet,SEAT,ONEBOX,Norwegian,Visyon (Estudio Bay SL),Bluekik,Exoticca,CAPITOLE CONSULTING,Movetia,
8,Bit Genoma Digital Solutions SL,AKKA,Atomian,Project Lobster,Bit Genoma Digital Solutions,C14TORCE,The Hotels Network,ONEBOX,Volcanic Internet,SEAT,...,,,,,,,,,,InboundCycle
9,Visyon (Estudio Bay SL),Volcanic Internet,Norwegian,Exoticca,Movetia,CAPITOLE CONSULTING,,,,,...,Bit Genoma Digital Solutions SL,Project Lobster,C14TORCE,Bit Genoma Digital Solutions,Farmaconnect,Atomian,The Hotels Network,ONEBOX,SEAT,Bluekik


True

True

True

...

True

True

True

True

True

# Pruebas